### Данный notebook предназначен для запуска на jupyter server внутри AICloud. Если вы хотит провести те же самые операции пользуясь своим удаленным jupyter сервером, то посмотрите пример отправки задач на обучение REST.

# 0. Подключение библиотеки для работы с кластером и сервисом

In [24]:
try:
    import client_lib
except ImportError:
    raise RuntimeError("Скрипт не предназначен для запуска вне кластера")

In [2]:
import requests

In [4]:
def save_file(url, filename):
    # Download file and place it on local storage
    r = requests.get(url)

    with open(filename, 'wb') as f:
        f.write(r.content)
    print(f"{filename} downloaded from {url}")

# 1. Размещаем на локальные диски модель и данные

По умолчанию локальная рабочая директория установлена в папку `/home/jovyan`

In [5]:
save_file("https://github.com/sbercloud-ai/aicloud-examples/raw/master/quick-start/mnist.npz", "mnist.npz")

mnist.npz downloaded from https://github.com/sbercloud-ai/aicloud-examples/raw/master/quick-start/mnist.npz


In [9]:
save_file("https://github.com/sbercloud-ai/aicloud-examples/raw/master/quick-start/requirements.txt", "requirements.txt")

requirements.txt downloaded from https://github.com/sbercloud-ai/aicloud-examples/raw/master/quick-start/requirements.txt


In [10]:
save_file("https://raw.githubusercontent.com/sbercloud-ai/aicloud-examples/master/quick-start/tensorflow_mnist_estimator.py", "tensorflow_mnist_estimator.py")

tensorflow_mnist_estimator.py downloaded from https://raw.githubusercontent.com/sbercloud-ai/aicloud-examples/master/quick-start/tensorflow_mnist_estimator.py


# 2. Сборка образа с нужными библиотеками

##### Файлик requirements.txt и его содержимое

In [11]:
!cat /home/jovyan/requirements.txt

tensorflow-gpu==1.15.0

##### Запуск сборки кастомного образа с необходимыми библиотеками

In [7]:
job = client_lib.ImageBuildJob(
                    from_image='registry.aicloud.sbcp.ru/horovod-tf15',
                    requirements_file='/home/jovyan/requirements.txt'
)

In [8]:
job.submit()

'ImageBuildJob "{\'image\': \'registry.aicloud.sbcp.ru/ai0000001-0090/365cf0ee-af91-4b62-865f-f0ce28a9efef\', \'name\': \'image-build-job-zdvgd\', \'status\': \'ok\'}" created'

In [9]:
job.new_image

'registry.aicloud.sbcp.ru/ai0000001-0090/365cf0ee-af91-4b62-865f-f0ce28a9efef'

In [10]:
job.logs()

INFO[0000] Resolved base name registry.aicloud.sbcp.ru/horovod-tf15 to registry.aicloud.sbcp.ru/horovod-tf15 
INFO[0000] Resolved base name registry.aicloud.sbcp.ru/horovod-tf15 to registry.aicloud.sbcp.ru/horovod-tf15 
INFO[0000] Retrieving image manifest registry.aicloud.sbcp.ru/horovod-tf15 
INFO[0000] Retrieving image manifest registry.aicloud.sbcp.ru/horovod-tf15 
INFO[0000] Built cross stage deps: map[]                
INFO[0000] Retrieving image manifest registry.aicloud.sbcp.ru/horovod-tf15 
INFO[0000] Retrieving image manifest registry.aicloud.sbcp.ru/horovod-tf15 
INFO[0000] Checking for cached layer registry.aicloud.sbcp.ru/ai0000001-0090/365cf0ee-af91-4b62-865f-f0ce28a9efef/cache:81d726fa7182b3cde5d942f1ea73c94d79b465af30c67d2382b18ddb6c0681c9... 
INFO[0000] No cached layer found for cmd RUN if [ -e /context/requirements.txt ]; then pip install --user --no-cache -r /context/requirements.txt; fi 
INFO[0000] Unpacking rootfs as cmd RUN if [ -e /context/requirements.txt ]; the

# 3. Запуск задачи обучения

#### Создание задачи и отправка на кластер

In [11]:
mnist_tf_run = client_lib.Job(base_image='registry.aicloud.sbcp.ru/ai0000001-0090/09aa1302-6d95-4f04-8f23-71013679aa00',
                              script='tensorflow_mnist_estimator.py', 
                              n_workers=2, n_gpus=4, warm_cache=False, detached=False)

#### В предыдущей строке мы указали образ, в рамках которого будет исполняться задача (base_image), скрипт, который будет запущен (script), а также количество хостовых исполнителей (n_workers) и количество GPU-карт на каждом (n_gpus). 
#### В этом случае под задачу будет запущено 2 узла по 4 ГПУ карты (ИТОГО - 8 GPU).
#### Теперь вызовем на объекте метод .submit() и отправим задачу в очередь исполнения на кластере

In [12]:
mnist_tf_run.submit()

'Job "lm-mpi-job-a4965598-9d4f-40ab-b001-31e609aeb3f9" created'

#### Вы можете просмотреть список ваших задач и их статус с помощью следующего метода

In [13]:
client_lib.jobs()

2020-02-28T14:54:31Z : lm-mpi-job-0170cd13-0dde-4741-9b02-ce9f0eee1349 : Completed
2020-02-20T08:39:16Z : lm-mpi-job-0ac8ff95-7c6d-4c20-bf4f-19cdec0070ca : Completed
2020-03-05T08:17:51Z : lm-mpi-job-0dbfcf60-ebf4-4020-bc82-d93862d597b4 : Completed
2020-03-03T13:28:02Z : lm-mpi-job-0e875e8d-01b1-4ed9-a4a4-6d4de951fb00 : Completed
2020-03-03T15:46:37Z : lm-mpi-job-1702cf84-60f1-4ca9-bba6-a22605c09558 : Completed
2020-03-03T13:33:09Z : lm-mpi-job-27d99673-f827-441a-994c-7cfb4e611679 : Completed
2020-02-13T16:39:04Z : lm-mpi-job-335a09be-1ecb-4661-920b-857d21e6f850 : Completed
2020-02-04T09:58:29Z : lm-mpi-job-4ef73dd8-9d3d-487b-9570-7d66df018ef6 : Completed
2020-02-04T09:40:27Z : lm-mpi-job-53783174-5696-4f40-a5ff-bba8dd5c3963 : Completed
2020-02-27T15:42:14Z : lm-mpi-job-5801b303-f94f-41b8-a928-dca1cc55a068 : Completed
2020-02-27T10:38:49Z : lm-mpi-job-5f7e27d6-8391-4ccc-9bb8-c39a5b7928fe : Completed
2020-02-13T14:25:28Z : lm-mpi-job-73cdd281-f0f1-48a1-83cc-74930836cf54 : Completed
2020

#### Логи задачи

In [ ]:
mnist_tf_run.logs()

##### или по названию задачи

In [7]:
client_lib.logs("lm-mpi-job-5a371168-5ed5-4ec6-8af0-490c8cc4e167")

Job in queue. Try later

#### (ЕСЛИ НЕОБХОДИМО ПРЕРВАТЬ ЗАДАЧУ) Варианты убийства задачи

In [10]:
mnist_tf_run.kill()

'Job "lm-mpi-job-a0f7fdc7-3b27-45aa-a709-4f4821b887a1" deleted'

In [38]:
client_lib.kill('lm-mpi-job-3f332e07-ebca-4e22-afc8-852d6c53a922')

'Job "lm-mpi-job-3f332e07-ebca-4e22-afc8-852d6c53a922" deleted'

# 4. Сохранение результатов обучения модели

В процессе обучения модели чекпоинты сохраняются в папке `/home/jovyan/checkpoints_tf/mnist_convnet_model/`, их можно скачать:
* через веб-интерфейс jupyter notebook
* скопировать из локально доступной файловой системы на s3

## Выгрузка результатов на S3

##### Указываем данные для доступа к S3 бакету

Далее вам необходимо указать данные для доступа к S3-хранилищу в вашем аккаунте (которые можно найти на портале в "моих услугах"->"AI Cloud").

Введите внутрь одинарных кавычек `''` по порядку свои : `S3 namespace`, `S3 access key`, `S3 security key`

In [15]:
# Specify S3 credentials as follows
# save_aws_credentials('b175178b-f537-4a1c-8977-7871bb9dc448-namespace', 'b13f3480-a701-48fb-85c5-b4a3b36f3713', 'BPBdF5E9CGAFv+rmBakFc+R0krVCHd/u4rPsqfG8')
client_lib.save_aws_credentials('', '', '')

Ниже укажем s3-bucket, туда будут копироваться файлы

In [ ]:
# Specify S3 bucket
# s3_bucket = "s3://9b8d0e96-6461-47cf-9507-21ede838c9ac-bucket/"
s3_bucket = ""

In [ ]:
# Перемещение отдельных файлов
# client_lib.S3CopyJob("/home/jovyan/checkpoints_tf/mnist_convnet_model/model.ckpt-10937.data-00000-of-00001", s3_bucket).submit()
# client_lib.S3CopyJob("/home/jovyan/checkpoints_tf/mnist_convnet_model/model.ckpt-10937.index", s3_bucket).submit()
# client_lib.S3CopyJob("/home/jovyan/checkpoints_tf/mnist_convnet_model/model.ckpt-10937.meta", s3_bucket).submit()

In [19]:
# Перемещение папки с файлами
client_lib.S3CopyJob("/home/jovyan/checkpoints_tf/mnist_convnet_model/", s3_bucket, recursive=True).submit()

'S3CopyJob "{\'name\': \'copying-job-wvgz2\', \'status\': \'ok\'}" created'

Если все было сделано правильно, то Чекпоинт сохранится на ваш S3, и из него можно будет восстановить объект с моделью